In [4]:
import os 
from llama_index.llms.groq import Groq

C:\Users\salaa\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
from groq import Groq

# Initialize the Groq client with the API key
llm = Groq(api_key="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")

def assistive_merge(paragraphs):
    merge_prompt = "\n%%%\n".join(paragraphs)
    response = llm.chat.completions.create(
        model="mixtral-8x7b-32768",
        messages=[
            {"role": "system", "content": "You area helpful assistant."},
            {"role": "user", "content": f"Merge the following text segments. The first and second text are delimited by (%%%):\n{merge_prompt}"}
        ]
    )
    merged_text = response.choices[0].message.content.strip()
    return merged_text

# Example chopped text
paragraph1 = "Alice, the CEO of Acme Corp, announced that the company will a"
paragraph2 = "wire Widget Inc. This acquisition is expected to enhance Acme's product line and market reach."

# Use the assistive merge function to combine the paragraphs
merged_text = assistive_merge([paragraph1, paragraph2])
print("Merged Text:", merged_text)


Merged Text: Alice, the CEO of Acme Corp, announced that the company will acquire Widget Inc. This acquisition is expected to enhance Acme's product line and market reach.


In [51]:
from groq import Groq

# Initialize the Groq client with the API key
llm = Groq(api_key="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")

def assistive_merge(paragraphs):
    merge_prompt = "\n%%%\n".join(paragraphs)
    response = llm.chat.completions.create(
        model="mixtral-8x7b-32768",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Merge the following text segments. The first and second text are delimited by (%%%):\n{merge_prompt}"}
        ]
    )
    merged_text = response.choices[0].message.content.strip()
    return merged_text

def extract_entities_and_relationships(merged_text):
    response = llm.chat.completions.create(
        model="mixtral-8x7b-32768",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": (
                "Extract entities and relationships from the following text:\n"
                f"{merged_text}\n"
                "Please provide the output strictly in the following format with no variance:\n"
                "1. Source Entity: <source entity>\n"
                "2. Relationship: <relationship>\n"
                "3. Destination Entity: <destination entity>\n"
                "Ensure that each triplet is listed in the exact same format for consistency."
            )}
        ]
    )
    structured_response = response.choices[0].message.content.strip()
    triplets = parse_entities_and_relationships(structured_response)
    return triplets

def parse_entities_and_relationships(structured_response):
    lines = structured_response.split("\n")
    triplets = []
    current_triplet = {}
    
    for line in lines:
        if line.startswith("1. Source Entity:"):
            if current_triplet:
                triplets.append(current_triplet)
                current_triplet = {}
            current_triplet['source'] = line.replace("1. Source Entity:", "").strip()
        elif line.startswith("2. Relationship:"):
            current_triplet['relationship'] = line.replace("2. Relationship:", "").strip()
        elif line.startswith("3. Destination Entity:"):
            current_triplet['destination'] = line.replace("3. Destination Entity:", "").strip()
    
    if current_triplet:
        triplets.append(current_triplet)
    
    return triplets

# Example usage
paragraph1 = "Alice, the CEO of Acme Corp, announced that the company will a"
paragraph2 = "wire Widget Inc. This acquisition is expected to enhance Acme's product line and market reach."

# Use the assistive merge function to combine the paragraphs
merged_text = assistive_merge([paragraph1, paragraph2])
print("Merged Text:", merged_text)

# Extract entities and relationships from the merged text
entities_and_relationships = extract_entities_and_relationships(merged_text)
print("Entities and Relationships:")
for triplet in entities_and_relationships:
    print(f"Source: {triplet['source']}, Relationship: {triplet['relationship']}, Destination: {triplet['destination']}")

Merged Text: Alice, the CEO of Acme Corp, announced that the company will acquire Widget Inc. This acquisition is expected to enhance Acme's product line and market reach.
Entities and Relationships:
Source: Alice, Relationship: is the CEO of, Destination: Acme Corp
Source: Acme Corp, Relationship: will acquire, Destination: Widget Inc
Source: The acquisition of Widget Inc, Relationship: is expected to enhance, Destination: Acme Corp's product line and market reach


In [52]:
# Initial sentences
paragraph1 = "Alice, the CEO of Acme Corp, announced that the company will acquire Widget Inc."
paragraph2 = "This acquisition is expected to enhance Acme's product line and market reach."

# Use the assistive merge function to combine the paragraphs
merged_text1 = assistive_merge([paragraph1, paragraph2])
print("Merged Text:", merged_text1)

# Extract entities and relationships from the merged text
entities_and_relationships1 = extract_entities_and_relationships(merged_text1)
print("Entities and Relationships (Initial):")
for triplet in entities_and_relationships1:
    print(f"Source: {triplet['source']}, Relationship: {triplet['relationship']}, Destination: {triplet['destination']}")
print("\nExisting Entities:", existing_entities)
print("Existing Relationships:", existing_relationships)


Merged Text: Alice, the CEO of Acme Corp, announced that the company will acquire Widget Inc., an acquisition expected to enhance Acme's product line and market reach.
Entities and Relationships (Initial):
Source: Alice, Relationship: is, Destination: CEO of Acme Corp
Source: Acme Corp, Relationship: will acquire, Destination: Widget Inc
Source: The acquisition of Widget Inc, Relationship: is expected to, Destination: Enhance Acme's product line and market reach

Existing Entities: {'alice', 'ceo of acme corp'}
Existing Relationships: [('alice', 'Is', 'ceo of acme corp')]


In [53]:
# Additional sentence
paragraph3 = "Bob, the CTO of Acme Corp, confirmed that the integration with Widget Inc. will begin next quarter."

# Use the assistive merge function to combine the new sentence with the previous merged text
merged_text2 = assistive_merge([merged_text1, paragraph3])
print("Merged Text (After Adding New Sentence):", merged_text2)

# Extract entities and relationships from the new merged text
entities_and_relationships2 = extract_entities_and_relationships(merged_text2)
print("Entities and Relationships (After Adding New Sentence):")
for triplet in entities_and_relationships2:
    print(f"Source: {triplet['source']}, Relationship: {triplet['relationship']}, Destination: {triplet['destination']}")
print("\nExisting Entities (After Adding New Sentence):", existing_entities)
print("Existing Relationships (After Adding New Sentence):", existing_relationships)


Merged Text (After Adding New Sentence): Alice, the CEO of Acme Corp, announced that the company will acquire Widget Inc., an acquisition expected to enhance Acme's product line and market reach. Bob, the CTO of Acme Corp, confirmed that the integration with Widget Inc. will begin next quarter.
Entities and Relationships (After Adding New Sentence):
Source: Alice, Relationship: is the CEO of, Destination: Acme Corp
Source: Acme Corp, Relationship: will acquire, Destination: Widget Inc.
Source: the acquisition of Widget Inc. by Acme Corp, Relationship: is expected to enhance, Destination: Acme Corp's product line and market reach
Source: Bob, Relationship: is the CTO of, Destination: Acme Corp
Source: Acme Corp, Relationship: will begin integration with, Destination: Widget Inc.
Source: the integration of Acme Corp with, Relationship: will start, Destination: Widget Inc.
Source: the integration of Acme Corp with Widget Inc., Relationship: is scheduled to begin, Destination: next quarter

In [57]:
from groq import Groq

# Initialize the Groq client with the API key
llm = Groq(api_key="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")

existing_entities = set()
existing_relationships = []

def assistive_merge(paragraphs):
    merge_prompt = "\n%%%\n".join(paragraphs)
    response = llm.chat.completions.create(
        model="mixtral-8x7b-32768",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Merge the following text segments. The first and second text are delimited by (%%%):\n{merge_prompt}"}
        ]
    )
    merged_text = response.choices[0].message.content.strip()
    return merged_text

def extract_entities_and_relationships(merged_text):
    response = llm.chat.completions.create(
        model="mixtral-8x7b-32768",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": (
                "Extract entities and relationships from the following text:\n"
                f"{merged_text}\n"
                "Please provide the output strictly in the following format with no variance:\n"
                "1. Source Entity: <source entity>\n"
                "2. Relationship: <relationship>\n"
                "3. Destination Entity: <destination entity>\n"
                "Ensure that each triplet is listed in the exact same format for consistency."
            )}
        ]
    )
    structured_response = response.choices[0].message.content.strip()
    triplets = parse_entities_and_relationships(structured_response)
    update_entities_and_relationships(triplets)
    return triplets

def parse_entities_and_relationships(structured_response):
    lines = structured_response.split("\n")
    triplets = []
    current_triplet = {}
    
    for line in lines:
        if line.startswith("1. Source Entity:"):
            if current_triplet and all(key in current_triplet for key in ['source', 'relationship', 'destination']):
                triplets.append(current_triplet)
                current_triplet = {}
            current_triplet['source'] = line.replace("1. Source Entity:", "").strip()
        elif line.startswith("2. Relationship:"):
            current_triplet['relationship'] = line.replace("2. Relationship:", "").strip()
        elif line.startswith("3. Destination Entity:"):
            current_triplet['destination'] = line.replace("3. Destination Entity:", "").strip()
    
    if current_triplet and all(key in current_triplet for key in ['source', 'relationship', 'destination']):
        triplets.append(current_triplet)
    
    return triplets

def normalize_entity_name(entity):
    # Example normalization function; you can expand this as needed
    entity = entity.lower().strip()
    return entity

def update_entities_and_relationships(triplets):
    global existing_entities, existing_relationships
    for triplet in triplets:
        try:
            # Normalize entity names
            source = normalize_entity_name(triplet['source'])
            destination = normalize_entity_name(triplet['destination'])
            
            # Check and update entities
            if source not in existing_entities:
                existing_entities.add(source)
            if destination not in existing_entities:
                existing_entities.add(destination)
            
            # Check and update relationships
            relationship = (source, triplet['relationship'], destination)
            if relationship not in existing_relationships:
                existing_relationships.append(relationship)
        except KeyError as e:
            print(f"Missing key in triplet: {e}")

# Example usage
paragraph1 = "Alice, the CEO of Acme Corp, announced that the company will acquire Widget Inc."
paragraph2 = "This acquisition is expected to enhance Acme's product line and market reach."

# Use the assistive merge function to combine the paragraphs
merged_text = assistive_merge([paragraph1, paragraph2])
print("Merged Text:", merged_text)

# Extract entities and relationships from the merged text
entities_and_relationships = extract_entities_and_relationships(merged_text)
print("Entities and Relationships (Initial):")
for triplet in entities_and_relationships:
    print(f"Source: {triplet['source']}, Relationship: {triplet['relationship']}, Destination: {triplet['destination']}")

# Print the existing entities and relationships
print("\nExisting Entities:", existing_entities)
print("Existing Relationships:", existing_relationships)

# Adding a new sentence to check for consistency
paragraph3 = "Bob, the CTO of Acme Corp, confirmed that the integration with Widget Inc. will begin next quarter."

# Use the assistive merge function to combine the new sentence with the previous merged text
merged_text2 = assistive_merge([merged_text, paragraph3])
print("Merged Text (After Adding New Sentence):", merged_text2)

# Extract entities and relationships from the new merged text
entities_and_relationships2 = extract_entities_and_relationships(merged_text2)
print("Entities and Relationships (After Adding New Sentence):")
for triplet in entities_and_relationships2:
    print(f"Source: {triplet['source']}, Relationship: {triplet['relationship']}, Destination: {triplet['destination']}")

# Print the existing entities and relationships after adding new sentence
print("\nExisting Entities (After Adding New Sentence):", existing_entities)
print("Existing Relationships (After Adding New Sentence):", existing_relationships)

Merged Text: Alice, the CEO of Acme Corp, announced that the company will acquire Widget Inc., an acquisition expected to enhance Acme's product line and market reach.
Entities and Relationships (Initial):
Source: Alice, Relationship: is, Destination: CEO of Acme Corp
Source: Acme Corp, Relationship: will acquire, Destination: Widget Inc
Source: The acquisition of Widget Inc, Relationship: is expected to, Destination: Enhance Acme's product line
Source: The acquisition of Widget Inc, Relationship: is expected to, Destination: Expand Acme's market reach

Existing Entities: {'acme corp', 'alice', 'ceo of acme corp', "enhance acme's product line", 'the acquisition of widget inc', "expand acme's market reach", 'widget inc'}
Existing Relationships: [('alice', 'is', 'ceo of acme corp'), ('acme corp', 'will acquire', 'widget inc'), ('the acquisition of widget inc', 'is expected to', "enhance acme's product line"), ('the acquisition of widget inc', 'is expected to', "expand acme's market reach

In [83]:
from groq import Groq

existing_entities = set()
existing_relationships = []

llm = Groq(api_key="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")

In [84]:
def assistive_merge(paragraphs):
    merge_prompt = "\n%%%\n".join(paragraphs)
    response = llm.chat.completions.create(
        model="mixtral-8x7b-32768",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Merge the following text segments. The first and second text are delimited by (%%%):\n{merge_prompt}"}
        ]
    )
    merged_text = response.choices[0].message.content.strip()
    return merged_text

In [97]:
def extract_entities_and_relationships(merged_text):
    response = llm.chat.completions.create(
        model="mixtral-8x7b-32768",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": (
                "Extract entities and relationships from the following text:\n"
                f"{merged_text}\n"
                "Avoid redundant relationship triplets. Separate entities and create new entities if needed.\n"
                "Current entities:\n" + "\n".join(existing_entities) + "\n"
                "Current relationships:\n" + "\n".join(f"{r[0]} - {r[1]} - {r[2]}" for r in existing_relationships) + "\n"
                "Please provide the output strictly in the following format with no variance:\n"
                "1. Source Entity: <source entity>\n"
                "2. Relationship: <relationship>\n"
                "3. Destination Entity: <destination entity>\n"
                "Ensure that each triplet is listed in the exact same format for consistency."
            )}
        ]
    )
    structured_response = response.choices[0].message.content.strip()
    triplets = parse_entities_and_relationships(structured_response)
    update_entities_and_relationships(triplets)
    return triplets

def parse_entities_and_relationships(structured_response):
    lines = structured_response.split("\n")
    triplets = []
    current_triplet = {}
    
    for line in lines:
        if line.startswith("1. Source Entity:"):
            if current_triplet and all(key in current_triplet for key in ['source', 'relationship', 'destination']):
                triplets.append(current_triplet)
                current_triplet = {}
            current_triplet['source'] = normalize_entity_name(line.replace("1. Source Entity:", "").strip())
        elif line.startswith("2. Relationship:"):
            current_triplet['relationship'] = line.replace("2. Relationship:", "").strip()
        elif line.startswith("3. Destination Entity:"):
            current_triplet['destination'] = normalize_entity_name(line.replace("3. Destination Entity:", "").strip())
    
    if current_triplet and all(key in current_triplet for key in ['source', 'relationship', 'destination']):
        triplets.append(current_triplet)
    
    return triplets

def normalize_entity_name(entity):
    entity = entity.lower().strip()
    entity = entity.replace("inc.", "inc").replace("corp.", "corp")
    return entity

def update_entities_and_relationships(triplets):
    global existing_entities, existing_relationships
    for triplet in triplets:
        try:
            source = normalize_entity_name(triplet['source'])
            destination = normalize_entity_name(triplet['destination'])
            
            if source not in existing_entities:
                existing_entities.add(source)
            if destination not in existing_entities:
                existing_entities.add(destination)
            
            relationship = (source, triplet['relationship'], destination)
            if relationship not in existing_relationships:
                existing_relationships.append(relationship)
        except KeyError as e:
            print(f"Missing key in triplet: {e}")


In [99]:
# Initial sentences
paragraph1 = "Alice, the CEO of Acme Corp, announced that the company will acquire Widget Inc."
paragraph2 = "This acquisition is expected to enhance Acme's product line and market reach."

merged_text = assistive_merge([paragraph1, paragraph2])
print("Merged Text:", merged_text)

entities_and_relationships = extract_entities_and_relationships(merged_text)
print("Entities and Relationships (Initial):")
for triplet in entities_and_relationships:
    print(f"Source: {triplet['source']}, Relationship: {triplet['relationship']}, Destination: {triplet['destination']}")

print("\nExisting Entities:", existing_entities)
print("Existing Relationships:", existing_relationships)

# New sentence
paragraph3 = "Bob, the CTO of Acme Corp, confirmed that the integration with Widget Inc. will begin next quarter."

merged_text2 = assistive_merge([merged_text, paragraph3])
print("Merged Text (After Adding New Sentence):", merged_text2)

entities_and_relationships2 = extract_entities_and_relationships(merged_text2)
print("Entities and Relationships (After Adding New Sentence):")
for triplet in entities_and_relationships2:
    print(f"Source: {triplet['source']}, Relationship: {triplet['relationship']}, Destination: {triplet['destination']}")

print("\nExisting Entities (After Adding New Sentence):", existing_entities)
print("Existing Relationships (After Adding New Sentence):", existing_relationships)


Merged Text: Alice, the CEO of Acme Corp, announced that the company will acquire Widget Inc., an acquisition expected to enhance Acme's product line and market reach.
Entities and Relationships (Initial):
Source: alice, Relationship: is the CEO of, Destination: acme corp

Existing Entities: {'acme corp', 'product line and market reach (as an attribute)', 'alice', 'product line and market reach', 'product line and market reach (as an attribute of acme corp)', 'bob', 'widget inc', 'enhancement (with "product line and market reach" as attributes)'}
Existing Relationships: [('alice', 'CEO of', 'acme corp'), ('acme corp', 'will acquire', 'widget inc'), ('acme corp', 'expected to enhance', 'product line and market reach'), ('bob', 'CTO of', 'acme corp'), ('acme corp', 'integration will begin', 'widget inc'), ('alice', 'is the CEO of', 'acme corp'), ('widget inc', 'will be acquired by', 'acme corp'), ('acme corp', 'is expected to have enhanced', 'product line and market reach (as an attribut

In [165]:
from llama_index.llms.groq import Groq
from llama_index.core.llms import ChatMessage

# Initialize the Groq model with the specified model
llm = Groq(model="mixtral-8x7b-32768", api_key="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")


In [237]:
def assistive_merge(paragraphs):
    merge_prompt = "\n%%%\n".join(paragraphs)
    response = llm.chat(
        messages=[
            ChatMessage(role="system", content="You are a helpful assistant."),
            ChatMessage(role="user", content=f"Merge the following text segments delimited by (%%%):\n{merge_prompt}")
        ]
    )
    # Extracting the actual content from the response message
    merged_text = response.message.content.strip()
    return merged_text

In [255]:
def extract_entities_and_relationships(merged_text):
    response = llm.chat(
        messages=[
            ChatMessage(role="system", content="You are a helpful assistant."),
            ChatMessage(role="user", content=(
                "Extract new entities and relationships from the following text:\n"
                f"{merged_text}\n"
                "Avoid redundant relationship triplets. Separate entities and create new entities if needed.\n"
                "Please provide the output strictly in the following format with no variance:\n"
                "1. Source Entity: <source entity>\n"
                "2. Relationship: <relationship>\n"
                "3. Destination Entity: <destination entity>\n"
                "Ensure each triplet is listed in the exact same format for consistency."
            ))
        ]
    )
    structured_response = response.message.content.strip()
    triplets = parse_entities_and_relationships(structured_response)
    update_entities_and_relationships(triplets)
    return triplets


In [256]:
def parse_entities_and_relationships(structured_response):
    lines = structured_response.split("\n")
    triplets = []
    current_triplet = {}

    for line in lines:
        if line.startswith("1. Source Entity:"):
            if current_triplet:
                triplets.append(current_triplet)
                current_triplet = {}
            current_triplet['source'] = normalize_entity_name(line.replace("1. Source Entity:", "").strip())
        elif line.startswith("2. Relationship:"):
            current_triplet['relationship'] = line.replace("2. Relationship:", "").strip()
        elif line.startswith("3. Destination Entity:"):
            current_triplet['destination'] = normalize_entity_name(line.replace("3. Destination Entity:", "").strip())

    if current_triplet:
        triplets.append(current_triplet)

    return triplets

def normalize_entity_name(entity):
    entity = entity.lower().strip()
    entity = entity.replace("inc.", "inc").replace("corp.", "corp")
    return entity




In [257]:
def update_entities_and_relationships(triplets):
    global existing_entities, existing_relationships
    for triplet in triplets:
        try:
            source = triplet['source']
            destination = triplet['destination']

            if source not in existing_entities:
                existing_entities.add(source)
            if destination not in existing_entities:
                existing_entities.add(destination)

            relationship = (source, triplet['relationship'], destination)
            if relationship not in existing_relationships:
                existing_relationships.append(relationship)
        except KeyError as e:
            print(f"Missing key in triplet: {e}")


In [258]:
# Initialize global variables
existing_entities = set()
existing_relationships = []

# Initial sentences
paragraph1 = "Alice, the CEO of Acme Corp, announced that the company will acquire Widget Inc."
paragraph2 = "This acquisition is expected to enhance Acme's product line and market reach."

merged_text = assistive_merge([paragraph1, paragraph2])
print("Merged Text:", merged_text)

entities_and_relationships = extract_entities_and_relationships(merged_text)
print("Entities and Relationships (Initial):")
for triplet in entities_and_relationships:
    print(f"Source: {triplet['source']}, Relationship: {triplet['relationship']}, Destination: {triplet['destination']}")

print("\nExisting Entities:", existing_entities)
print("Existing Relationships:", existing_relationships)

# New sentence
paragraph3 = "Bob, the CTO of Acme Corp, confirmed that the integration with Widget Inc. will begin next quarter."

merged_text2 = assistive_merge([merged_text, paragraph3])
print("Merged Text (After Adding New Sentence):", merged_text2)

entities_and_relationships2 = extract_entities_and_relationships(merged_text2)
print("Entities and Relationships (After Adding New Sentence):")
for triplet in entities_and_relationships2:
    print(f"Source: {triplet['source']}, Relationship: {triplet['relationship']}, Destination: {triplet['destination']}")

print("\nExisting Entities (After Adding New Sentence):", existing_entities)
print("Existing Relationships (After Adding New Sentence):", existing_relationships)


Merged Text: Alice, the CEO of Acme Corp, announced that the company will acquire Widget Inc, an acquisition expected to enhance Acme's product line and market reach.
Entities and Relationships (Initial):
Source: alice, Relationship: CEO of, Destination: acme corp

Existing Entities: {'acme corp', 'alice'}
Existing Relationships: [('alice', 'CEO of', 'acme corp')]
Merged Text (After Adding New Sentence): Alice, the CEO of Acme Corp, announced that the company will acquire Widget Inc, an acquisition expected to enhance Acme's product line and market reach. Bob, the CTO of Acme Corp, confirmed that the integration with Widget Inc. will begin next quarter.
Entities and Relationships (After Adding New Sentence):
Source: alice, Relationship: Is, Destination: ceo of acme corp

Existing Entities (After Adding New Sentence): {'acme corp', 'alice', 'ceo of acme corp'}
Existing Relationships (After Adding New Sentence): [('alice', 'CEO of', 'acme corp'), ('alice', 'Is', 'ceo of acme corp')]
